In [1]:
# Import libraries
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import geopy.distance
from geopy.distance import geodesic

In [2]:
# Read csv files
warehouse_csv = pd.read_csv('distr.csv')
entry_loc_csv = pd.read_csv('entry_loc.csv')

In [3]:
# Set up geolocator
geolocator = Nominatim(user_agent="user",timeout=10)

In [4]:
warehouse_loc = warehouse_csv.copy()
warehouse_loc['lat'] = np.nan
warehouse_loc['lon'] = np.nan

# Find coordinates for warehouses
for index, row in warehouse_loc.iterrows():
    location_results = geolocator.geocode(row['city']+', '+row['state'], exactly_one=False)
    if location_results is not None:
        row['lat'] = location_results[0][1][0]
        row['lon'] = location_results[0][1][1]
    warehouse_loc.loc[index] = row
      
warehouse_loc.to_csv('warehouse_loc.csv')

In [5]:
# Drop NaN observations
entry_loc_csv = entry_loc_csv.dropna(subset=['lat', 'lon'])

# Create a tuple for the coordinates
warehouse_loc['lat_long'] = warehouse_loc[['lat', 'lon']].apply(tuple, axis=1)
entry_loc_csv['lat_long'] = entry_loc_csv[['lat', 'lon']].apply(tuple, axis=1)

In [6]:
entry_loc_csv1 = entry_loc_csv.copy()

# Find difference between warehouses and markets
for i, row1 in warehouse_loc.iterrows(): 
    distances = []
    col_name = str(row1['store']) + " - " + str(row1['city'])
    for j, row2 in entry_loc_csv1.iterrows():
        distances.append(geopy.distance.geodesic(row2['lat_long'], row1['lat_long']).km)
    entry_loc_csv1[col_name] = distances
    distances.clear()

In [7]:
# Find minimum distance between each store and all warehouses
LOW_cols = entry_loc_csv1.columns[entry_loc_csv1.columns.str.contains('LOW - ')]
entry_loc_csv1['low warehouse distance'] = entry_loc_csv1[LOW_cols].min(axis=1)

HD_cols = entry_loc_csv1.columns[entry_loc_csv1.columns.str.contains('HD - ')]
entry_loc_csv1['hd warehouse distance'] = entry_loc_csv1[HD_cols].min(axis=1)

In [8]:
entry_loc_csv1.to_csv('entry_loc2.csv')